---
title: "Smol Models"
author: Deepam Minda
# date: "Sep 12, 2023"
# categories: [classification, few-shot learning, nlp]
# image: "cover.jpeg"
format:
  html:
    toc: true
    toc-depth: 3
    html-math-method: katex
    css: ../../styles.css
    code-fold: true
execute:
  warning: false
draft: true
---

In [98]:
# def find_sum

s = "0"
s.count('0')
counts_of_01s = [(0,0)]
counts_of_01s.extend([(s[:i+1].count('0'), s[:i+1].count('1')) for i in range(len(s))])
counts_of_01s

[(0, 0), (1, 0)]

In [100]:


s.count('0')
counts_of_01s = [(0,0)]
counts_of_01s.extend([(s[:i+1].count('0'), s[:i+1].count('1')) for i in range(len(s))])
counts_of_01s

ans=0
substrings = set()
for i in range(len(s)):
    for j in range(i, len(s)):
        print(f"i:{i}, j:{j}")
        if i==j: 
            ans+=1
            continue

        subs = s[i:j+1]
        if subs=="": continue
        if subs in substrings:
        
            continue
        
        substrings.add(subs)
        count_of_0s, count_of_1s = abs(counts_of_01s[i][0]-counts_of_01s[j+1][0]), abs(counts_of_01s[i][1]-counts_of_01s[j+1][1])
        if (count_of_0s == count_of_1s):
            ans+=2
        else:
            ans+=1
        # print(f"ans {ans}")
print(ans)
    



i:0, j:0
1
